In [1]:
# install package skfeature-chappers
%pip install skfeature-chappers

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/opt/tljh/user/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
#Import Library
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, KFold, cross_validate
from skfeature.function.similarity_based import fisher_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time

In [3]:
#import dataset
df = pd.read_csv('/home/jupyter-17523142/Rama/Dataset/dataset_rama.csv')
X = df.drop(['No', 'Mango Cultivars', 'Vit C (mg/100g)', 'TA (mg/100g)', 'SSC (oBrix)', 'label'], axis = 1)
y_vitc = df.loc[:, 'Vit C (mg/100g)']

# separate dataset into train and test
x_train, x_test, y_train, y_test = train_test_split(
    X, y_vitc, test_size = 0.3, random_state=0)

# Seleksi Fitur Fisher Score

In [4]:
%%time

#cari ranking setiap fitur berdasarkan fisher score
fisher_score = fisher_score.fisher_score(x_train.to_numpy(), y_train.to_numpy())
fisher_score

/home/jupyter-17523142/.local/lib/python3.7/site-packages/skfeature/function/similarity_based/fisher_score.py:59: RuntimeWarning: divide by zero encountered in true_divide
  score = 1.0 / lap_score - 1


CPU times: user 5.88 s, sys: 172 ms, total: 6.05 s
Wall time: 312 ms


array([ 778, 1022, 1031, ..., 1216, 1218,    0])

In [5]:
dir(fisher_score)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_finalize__',
 '__array_function__',
 '__array_interface__',
 '__array_prepare__',
 '__array_priority__',
 '__array_struct__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__

In [6]:
# bikin dataframe buat nyimpan fitur beserta rankingnya
fisher_ranking = pd.DataFrame(fisher_score)
features = pd.DataFrame(x_train.columns)
fisher_ranking = pd.concat([features,fisher_ranking],axis=1)

# Assign the column name
fisher_ranking.columns = ["Features","Fisher_Ranking"]

# Print features score
print(fisher_ranking.nsmallest(100,columns="Fisher_Ranking"))

     Features  Fisher_Ranking
1556   2500.2               0
260    1111.3               1
1188   1845.4               2
1406   2184.3               3
1189   1846.7               4
...       ...             ...
795      1442              95
799    1445.2              96
1405   2182.4              97
804    1449.3              98
809    1453.3              99

[100 rows x 2 columns]


In [7]:
# Fitur yang diseleksi
print(fisher_ranking.nsmallest(100,columns="Fisher_Ranking").iloc[:,0].values.flatten().tolist())

['2500.2', '1111.3', '1845.4', '2184.3', '1846.7', '1852', '1853.3', '1857.3', '1863.9', '1838.8', '1816.9', '2197.2', '1820.7', '2204.7', '1837.5', '1865.3', '1905', '1896.7', '1913.5', '1893.9', '1872', '1874.7', '1869.3', '1882.9', '1885.6', '2189.8', '2178.8', '1891.1', '2210.3', '1744.8', '1751.8', '1723.9', '1726.2', '1732', '1733.1', '1734.3', '1763.8', '1789.3', '1798', '1804.3', '1805.5', '1806.8', '1809.3', '1777.1', '1768.6', '1769.8', '1781.9', '1785.6', '1814.4', '1923.4', '2085.9', '2102.8', '2107.9', '2109.6', '2101.1', '2082.5', '2067.6', '2116.5', '2155.2', '2151.6', '2077.5', '1974.7', '1936.3', '1946.5', '1982.2', '2000.6', '1714.8', '1715.9', '1494.4', '1496.1', '2188', '1489.2', '1478.2', '1479.9', '1482.4', '1488.4', '1538.7', '1527.8', '1511.8', '1490.9', '1425.4', '1426.9', '1428.5', '1430.1', '1431.7', '1406.8', '1414.5', '1419.9', '1455.8', '1458.2', '1462.3', '1464', '1465.6', '1469', '1440.4', '1442', '1445.2', '2182.4', '1449.3', '1453.3']


In [8]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)

# Linear Regression

In [9]:
%%time

metrics = {'rmse': 'neg_root_mean_squared_error',
               'mse': 'neg_mean_squared_error',
               'mae': 'neg_mean_absolute_error',
               'r2': 'r2'} 

n_feat = range(1, 101)
min_mse = 1000
min_rmse = 1000
min_mae = 1000

for nfeat in n_feat:
    print("==================================================")
    start_time = time.time()

    x_train_selected = x_train[fisher_ranking.nlargest(100,columns="Fisher_Ranking").iloc[0:nfeat, 0]]
    x_test_selected = x_test[fisher_ranking.nlargest(100,columns="Fisher_Ranking").iloc[0:nfeat, 0]]

    lr_model = LinearRegression()

    lr_model.fit(x_train_selected, y_train)
    y_pred_lr = lr_model.predict(x_test_selected)

    scores = cross_validate(lr_model, x_train_selected, y_train, scoring=metrics, cv=cv, return_train_score=True)

    print("MSE model Linear Regression data Train dengan " + str(nfeat) + " fitur: " 
          + str(abs(round(scores['train_mse'].mean(), 2))))
    print("RMSE model Linear Regression data Train dengan " + str(nfeat) + " fitur:"
          + str(abs(round(scores['train_rmse'].mean(), 2))))
    print("MAE model Linear Regression data Train dengan " + str(nfeat) + " fitur:"
          + str(abs(round(scores['train_mae'].mean(), 2))))
    print("R2 model Linear Regression data Train dengan " + str(nfeat) + " fitur:"
          + str(abs(round(scores['train_r2'].mean(), 2))))
    print("-------------------------")
    print("MSE model Linear Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_lr), 2))))
    print("RMSE model Linear Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_lr, squared = False), 2))))
    print("MAE model Linear Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_absolute_error(y_test, y_pred_lr), 2))))
    print("R2 model Linear Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(r2_score(y_test, y_pred_lr), 2))))
    print(" ")
    
    if(abs(round(mean_squared_error(y_test, y_pred_lr), 2) < min_mse)):
        min_mse = abs(round(mean_squared_error(y_test, y_pred_lr), 2))

    if(abs(round(mean_squared_error(y_test, y_pred_lr, squared = False), 2) < min_rmse)):
        min_rmse = abs(round(mean_squared_error(y_test, y_pred_lr, squared = False), 2))

    if(abs(round(mean_absolute_error(y_test, y_pred_lr), 2) < min_mae)):
        min_mae = abs(round(mean_absolute_error(y_test, y_pred_lr), 2))

    end_time = time.time()
    print("Total waktu: ", end_time - start_time)

print("====Nilai Performa Tertinggi====")
print("Nilai MSE model terendah: " + str(min_mse))
print("Nilai RMSE model terendah: " + str(min_rmse))
print("Nilai MAE model terendah: " + str(min_mae))

MSE model Linear Regression data Train dengan 1 fitur: 188.99
RMSE model Linear Regression data Train dengan 1 fitur:13.75
MAE model Linear Regression data Train dengan 1 fitur:11.24
R2 model Linear Regression data Train dengan 1 fitur:0.01
-------------------------
MSE model Linear Regression data Test dengan 1 fitur:167.21
RMSE model Linear Regression data Test dengan 1 fitur:12.93
MAE model Linear Regression data Test dengan 1 fitur:9.91
R2 model Linear Regression data Test dengan 1 fitur:0.03
 
Total waktu:  0.12248682975769043
MSE model Linear Regression data Train dengan 2 fitur: 188.73
RMSE model Linear Regression data Train dengan 2 fitur:13.74
MAE model Linear Regression data Train dengan 2 fitur:11.23
R2 model Linear Regression data Train dengan 2 fitur:0.02
-------------------------
MSE model Linear Regression data Test dengan 2 fitur:169.48
RMSE model Linear Regression data Test dengan 2 fitur:13.02
MAE model Linear Regression data Test dengan 2 fitur:9.96
R2 model Linear R

# Random Forest Regression (n = 100)

In [10]:
%%time

metrics = {'rmse': 'neg_root_mean_squared_error',
               'mse': 'neg_mean_squared_error',
               'mae': 'neg_mean_absolute_error',
               'r2': 'r2'} 

n_feat = range(1, 101)
n_trees = [100]
min_mse = 1000
min_rmse = 1000
min_mae = 1000

for nfeat in n_feat:
    for ntrees in n_trees:
        print("==================================================")
        start_time = time.time()

        x_train_selected = x_train[fisher_ranking.nlargest(100,columns="Fisher_Ranking").iloc[0:nfeat, 0]]
        x_test_selected = x_test[fisher_ranking.nlargest(100,columns="Fisher_Ranking").iloc[0:nfeat, 0]]

        rfg_model = RandomForestRegressor(n_estimators=ntrees)
        
        rfg_model.fit(x_train_selected, y_train)
        y_pred_rfg=rfg_model.predict(x_test_selected)

        scores = cross_validate(rfg_model, x_train_selected, y_train, scoring=metrics, cv=cv, return_train_score=True)

        print("MSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_mse'].mean(), 2))))
        print("RMSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_rmse'].mean(), 2))))
        print("MAE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_mae'].mean(), 2))))
        print("R2 model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_r2'].mean(), 2))))
        print("-------------------------")
        print("MSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_squared_error(y_test, y_pred_rfg), 2))))
        print("RMSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))))
        print("MAE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))))
        print("R2 model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(r2_score(y_test, y_pred_rfg), 2))))
        print(" ")

        if(abs(round(mean_squared_error(y_test, y_pred_rfg), 2) < min_mse)):
            min_mse = abs(round(mean_squared_error(y_test, y_pred_rfg), 2))

        if(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2) < min_rmse)):
            min_rmse = abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))

        if(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2) < min_mae)):
            min_mae = abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))

        end_time = time.time()
        print("Total waktu: ", end_time - start_time)

print("====Nilai Performa Tertinggi====")
print("Nilai MSE model terendah: " + str(min_mse))
print("Nilai RMSE model terendah: " + str(min_rmse))
print("Nilai MAE model terendah: " + str(min_mae))

MSE model Random Forest Regression data Train dengan 1 fitur dan 100 trees:44.03
RMSE model Random Forest Regression data Train dengan 1 fitur dan 100 trees:6.63
MAE model Random Forest Regression data Train dengan 1 fitur dan 100 trees:5.16
R2 model Random Forest Regression data Train dengan 1 fitur dan 100 trees:0.77
-------------------------
MSE model Random Forest Regression data Test dengan 1 fitur dan 100 trees:268.02
RMSE model Random Forest Regression data Test dengan 1 fitur dan 100 trees:16.37
MAE model Random Forest Regression data Test dengan 1 fitur dan 100 trees:13.05
R2 model Random Forest Regression data Test dengan 1 fitur dan 100 trees:0.55
 
Total waktu:  1.2970633506774902
MSE model Random Forest Regression data Train dengan 2 fitur dan 100 trees:42.27
RMSE model Random Forest Regression data Train dengan 2 fitur dan 100 trees:6.5
MAE model Random Forest Regression data Train dengan 2 fitur dan 100 trees:5.06
R2 model Random Forest Regression data Train dengan 2 fit

# Random Forest Regression n = 150

In [11]:
%%time

metrics = {'rmse': 'neg_root_mean_squared_error',
               'mse': 'neg_mean_squared_error',
               'mae': 'neg_mean_absolute_error',
               'r2': 'r2'} 

n_feat = range(1, 101)
n_trees = [150]
min_mse = 1000
min_rmse = 1000
min_mae = 1000
for nfeat in n_feat:
    for ntrees in n_trees:
        print("==================================================")
        start_time = time.time()

        x_train_selected = x_train[fisher_ranking.nlargest(100,columns="Fisher_Ranking").iloc[0:nfeat, 0]]
        x_test_selected = x_test[fisher_ranking.nlargest(100,columns="Fisher_Ranking").iloc[0:nfeat, 0]]

        rfg_model = RandomForestRegressor(n_estimators=ntrees)
        
        rfg_model.fit(x_train_selected, y_train)
        y_pred_rfg=rfg_model.predict(x_test_selected)

        scores = cross_validate(rfg_model, x_train_selected, y_train, scoring=metrics, cv=cv, return_train_score=True)

        print("MSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_mse'].mean(), 2))))
        print("RMSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_rmse'].mean(), 2))))
        print("MAE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_mae'].mean(), 2))))
        print("R2 model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_r2'].mean(), 2))))
        print("-------------------------")
        print("MSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_squared_error(y_test, y_pred_rfg), 2))))
        print("RMSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))))
        print("MAE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))))
        print("R2 model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(r2_score(y_test, y_pred_rfg), 2))))
        print(" ")

        if(abs(round(mean_squared_error(y_test, y_pred_rfg), 2) < min_mse)):
            min_mse = abs(round(mean_squared_error(y_test, y_pred_rfg), 2))

        if(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2) < min_rmse)):
            min_rmse = abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))

        if(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2) < min_mae)):
            min_mae = abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))

        end_time = time.time()
        print("Total waktu: ", end_time - start_time)

print("====Nilai Performa Tertinggi====")
print("Nilai MSE model terendah: " + str(min_mse))
print("Nilai RMSE model terendah: " + str(min_rmse))
print("Nilai MAE model terendah: " + str(min_mae))

MSE model Random Forest Regression data Train dengan 1 fitur dan 150 trees:43.8
RMSE model Random Forest Regression data Train dengan 1 fitur dan 150 trees:6.61
MAE model Random Forest Regression data Train dengan 1 fitur dan 150 trees:5.16
R2 model Random Forest Regression data Train dengan 1 fitur dan 150 trees:0.77
-------------------------
MSE model Random Forest Regression data Test dengan 1 fitur dan 150 trees:268.56
RMSE model Random Forest Regression data Test dengan 1 fitur dan 150 trees:16.39
MAE model Random Forest Regression data Test dengan 1 fitur dan 150 trees:13.02
R2 model Random Forest Regression data Test dengan 1 fitur dan 150 trees:0.56
 
Total waktu:  1.7303147315979004
MSE model Random Forest Regression data Train dengan 2 fitur dan 150 trees:41.95
RMSE model Random Forest Regression data Train dengan 2 fitur dan 150 trees:6.47
MAE model Random Forest Regression data Train dengan 2 fitur dan 150 trees:5.04
R2 model Random Forest Regression data Train dengan 2 fit

# Random Forest Regression n = 200

In [12]:
%%time

metrics = {'rmse': 'neg_root_mean_squared_error',
               'mse': 'neg_mean_squared_error',
               'mae': 'neg_mean_absolute_error',
               'r2': 'r2'} 

n_feat = range(1, 101)
n_trees = [200]
min_mse = 1000
min_rmse = 1000
min_mae = 1000

for nfeat in n_feat:
    for ntrees in n_trees:
        print("==================================================")
        start_time = time.time()

        x_train_selected = x_train[fisher_ranking.nlargest(100,columns="Fisher_Ranking").iloc[0:nfeat, 0]]
        x_test_selected = x_test[fisher_ranking.nlargest(100,columns="Fisher_Ranking").iloc[0:nfeat, 0]]

        rfg_model = RandomForestRegressor(n_estimators=ntrees)
        
        rfg_model.fit(x_train_selected, y_train)
        y_pred_rfg=rfg_model.predict(x_test_selected)

        scores = cross_validate(rfg_model, x_train_selected, y_train, scoring=metrics, cv=cv, return_train_score=True)

        print("MSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_mse'].mean(), 2))))
        print("RMSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_rmse'].mean(), 2))))
        print("MAE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_mae'].mean(), 2))))
        print("R2 model Random Forest Regression data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:"
              + str(abs(round(scores['train_r2'].mean(), 2))))
        print("-------------------------")
        print("MSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_squared_error(y_test, y_pred_rfg), 2))))
        print("RMSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))))
        print("MAE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))))
        print("R2 model Random Forest Regression data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(abs(round(r2_score(y_test, y_pred_rfg), 2))))
        print(" ")

        if(abs(round(mean_squared_error(y_test, y_pred_rfg), 2) < min_mse)):
            min_mse = abs(round(mean_squared_error(y_test, y_pred_rfg), 2))

        if(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2) < min_rmse)):
            min_rmse = abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))

        if(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2) < min_mae)):
            min_mae = abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))

        end_time = time.time()
        print("Total waktu: ", end_time - start_time)

print("====Nilai Performa Tertinggi====")
print("Nilai MSE model terendah: " + str(min_mse))
print("Nilai RMSE model terendah: " + str(min_rmse))
print("Nilai MAE model terendah: " + str(min_mae))

MSE model Random Forest Regression data Train dengan 1 fitur dan 200 trees:43.85
RMSE model Random Forest Regression data Train dengan 1 fitur dan 200 trees:6.62
MAE model Random Forest Regression data Train dengan 1 fitur dan 200 trees:5.17
R2 model Random Forest Regression data Train dengan 1 fitur dan 200 trees:0.77
-------------------------
MSE model Random Forest Regression data Test dengan 1 fitur dan 200 trees:266.51
RMSE model Random Forest Regression data Test dengan 1 fitur dan 200 trees:16.33
MAE model Random Forest Regression data Test dengan 1 fitur dan 200 trees:12.95
R2 model Random Forest Regression data Test dengan 1 fitur dan 200 trees:0.55
 
Total waktu:  2.149155616760254
MSE model Random Forest Regression data Train dengan 2 fitur dan 200 trees:42.11
RMSE model Random Forest Regression data Train dengan 2 fitur dan 200 trees:6.48
MAE model Random Forest Regression data Train dengan 2 fitur dan 200 trees:5.06
R2 model Random Forest Regression data Train dengan 2 fit